In [ ]:
# Libraries 
# -----------------------------------------------------------------------
import pandas as pd
import numpy as np
# import sys
# import os
# sys.path.append(os.path.abspath(os.path.join('..', 'src'))) 


# Configuration
# -----------------------------------------------------------------------
pd.set_option('display.max_columns', None)


### Phase 1: Exploration and Cleanup

In [ ]:
# Pseudo code
# Exploratory Data Analysis
    # create dataframes DONE
    # at dataframe level 
        # print df name DONE
        # visual exploration with .head(), .tail() and sample() DONE
        # dataframe overview with .info() DONE
        # check duplicates with .duplicated().sum DONE
            # create separate dataframe with duplicates if needed DONE
        # check null values .isnull().sum() DONE
        # explore main statistical metrics for numerical and non numerical columns with .describe() and .describe(include = "object")
    # at column level
        # review columns names with .columns DONE
        # review unique values with .unique() DONE
        # review value count with .value_counts() DONE
        # check null values with .isnull().sum() DONE
        # atypical values
    # create function/s for the EDA
# Transformation
# Dataframe union
# Save data into a csv file


In [ ]:
# 📂 Open data 

df_customer_flights = pd.read_csv("data/customer_flight_activity.csv")
df_customer_loyalty = pd.read_csv("data/customer_loyalty_history.csv")
dataframes_dict = {"df_customer_flight✈️" : df_customer_flights, "df_customer_loyalty💞" : df_customer_loyalty}

In [ ]:
# ✍️ Exploratory Data Analysis functions definition

def explore_dataframes (df_dict):
    """
    Provides relevant information for a Exploratory Data Analysis.
    
    This function receives a dict with dataframes and executes a series of 
    functions aiming to provide relevant information to analize the data.

    Args:

    Returns:
    
    """
    duplicate_dataframes_dict = {}
    for name, df in df_dict.items():
        df_name = f"'{name.upper().replace('_',' ')}'"
        print(f" \n\n----------- DATAFRAME NAME: {df_name} -----------")
        print(f"\n{df_name} ---> Dataframe INFO:\n")
        display(df.info())
        print(f"\n{df_name} ---> FIRST FIVE (5) ROWS:")
        display(df.head())
        print(f"\n{df_name} ---> LAST FIVE (5) ROWS:")
        display(df.tail())
        print(f"\n{df_name} ---> SAMPLE (5) ROWS:")
        display(df.sample(5))
        print(f" \n{df_name} ---> DUPLICATES COUNT IS: {df.duplicated().sum()}, {round((df.duplicated().sum()/df.shape[0]*100),2)}% OVER TOTAL ROWS\n")
        if df.duplicated().sum() > 0:
            duplicates_df_name = name + "_duplicates"
            duplicates_df = df[df.duplicated(keep=False)]
            print(f"{df_name} ---> DATAFRAME WITH DUPLICATED ROWS (INCLUDING ALL APPEARANCES):\n")
            display(duplicates_df.head(10))
            duplicate_dataframes_dict[duplicates_df_name] = duplicates_df
        print(f"\n{df_name} --> COUNT OF ROWS WITH ALL NULL VALUES IS: {df.isnull().all(axis=1).sum()}\n")
        print(f"\n{df_name} --> COUNT OF COLUMNS WITH ALL NULL VALUES IS: {df.isnull().all().sum()}\n")
        print(F"\n{df_name} --> STATISTICAL METRICS FOR NUMERICAL COLUMNS:")
        display(df.describe().T)
        try:
            print(F"\n{df_name} --> STATISTICAL METRICS FOR CATEGORICAL COLUMNS:")
            display(df.describe(include="object").T)
        except:
            print("\nUPS... IT SEEMS LIKE THERE ARE NO COLUMNS WITH CATEGORICATL DATA")
    return duplicate_dataframes_dict

def explore_columns (df_dict):
    for name, df in df_dict.items():
        df_name = f"'{name.upper().replace('_',' ')}'"
        print(f" \n\n----------- DATAFRAME NAME: {df_name} -----------")
        for index, column in enumerate(df.columns):
            print (f"\n{index}) Column {column.upper()} (from {(df_name)} dataframe):")
            print (f"\n>>> UNIQUE VALUES:")
            print (df[column].unique())
            print (f"\n>>> VALUES COUNT:")
            print (df[column].value_counts())
            print (f"\n>>> COUNT OF DUPLICATES IN THE COLUMN:")
            print (df.duplicated(subset=[column]).sum())
            print (f"\n>>> COUNT OF NULL VALUES IN THE COLUMN:")
            print (df[column].isnull().sum())
            if df[column].dtype in ['int64', 'float64']:
                print("\nSTATISTICAL DESCRIPTION (NUMERIC):")
                display(df[column].describe())
            else:
                print("\nSTATISTICAL DESCRIPTION (CATEGORICAL):")
                display(df[column].describe(include='object'))
            print ("--------")

In [ ]:
# ▶️ Exploratory Data Analysis code execution

explore_dataframes(dataframes_dict)

explore_columns(dataframes_dict)


In [ ]:
# ✍️ Data transformation (cleaning and union) functions definition

def columns_to_snake_case (df_dict):
    for name, df in df_dict.items():
        df_name = f"'{name.upper().replace('_',' ')}'"
        df.columns = df.columns.str.lower().str.replace(' ', '_')
        print(f"\n{df_name} ---> Dataframe COLUMNS:\n")
        print(df.columns)

def impute_nulls_as_special_category(df, column_list, category_name):
    # Iterate through the list of columns to replace nulls with "Unknown"
    for column in column_list:
        if column in df.columns:
            # Replace nulls with the value "Unknown" for each column in the list
            df[column] = df[column].fillna(category_name)
            print (f"\nNull values imputed ✅ in column {column}.")
            print (f"UNIQUE VALUES:")
            print (df[column].unique())
        else:
            print(f"❌ The column '{column}' does not exist in the DataFrame.")

def transform_negative_values(df,column_list):
    """Transform negative values into its absolute value"""
    for column in column_list:
        if column in df.columns:
            df[column] = df[column].abs()
            print (f"\nNegative values transformed ✅ in column {column}.")
            print (f"UNIQUE VALUES:")
            print (df[column].unique())
        else:
            print(f"❌ The column '{column}' does not exist in the DataFrame.")

def impute_nulls_as_median(df, column_list):
    # Iterate through the list of columns to replace nulls with median
    for column in column_list:
        if column in df.columns:
            median= df[column].median()
            # Replace nulls with the median for each column in the list
            df[column] = df[column].fillna(median)
            print (f"\nNull values imputed ✅ in column {column}.")
            print (f"UNIQUE VALUES:")
            print (df[column].unique())
        else:
            print(f"❌ The column '{column}' does not exist in the DataFrame.")

def dfs_left_union (df_left, df_right):
    print (f"\nDataframe on the left's SHAPE:{df_left.shape}")
    print (f"\nDataframe on the right's SHAPE:{df_right.shape}")
    df_final = df_customer_flights.merge(df_customer_loyalty, how='left', on='loyalty_number')
    print("\nDatafrems were joined succesfully ✅")
    print (f"\nDataframe Final's SHAPE:{df_final.shape}")
    print (f"\nDataframe Final's COLUMNS are:{df_final.columns}")
    return df_final

In [ ]:
# ▶️ Data Transformation code execution

# Rename columns to snake case format
columns_to_snake_case(dataframes_dict)

# Delete records from df customer_flights that are duplicated in all columns, keeping only the first appearance
df_customer_flights.drop_duplicates(inplace=True)

print(f"DUPLICATES COUNT IS: {df_customer_flights.duplicated().sum()}")

# In columns "cancellation_year" and "cancelation_month" replace NaN with "Not Cancelled"
columns = ["cancellation_year","cancellation_month"]
category_name = "Not Cancelled"
impute_nulls_as_special_category(df_customer_loyalty,columns,category_name)

# In salary column transform negative value into positive and impute nulls assigning the median value
columns = ["salary"]
transform_negative_values(df_customer_loyalty,columns)
impute_nulls_as_median(df_customer_loyalty, columns)

# Join dataframes 
df_final = dfs_left_union(df_customer_flights,df_customer_loyalty)

# Save data into a csv
df_final.to_csv('data/customer_data_transformed.csv')

# Loyalty number column analysis

In [ ]:
def loyalty_number_analysis (df):
    
    # check the number of duplicates in loyalty_number column
    duplicates = df_customer_flights.duplicated(subset=['loyalty_number']).sum()
    print(f"There are {duplicates} duplicates in the column 'loyalty_number', which represent {round((duplicates/df_customer_flights.shape[0]*100),2)}% over the total")
    
    df_loyalty_number_duplicates = df_customer_flights[df_customer_flights.duplicated(subset=['loyalty_number'])]
    print (f"\n>>>DATAFRAME WITH loyalty_number:")
    display(df_loyalty_number_duplicates)
    # the information in the dataframe is organized in a way that there is one row per each year and each month within a year. 
    # Since there is info from 2 years it is expected to have 24 rows per loyalty_number (or less). So, I will only focus the analysis in those that appear more than 24 times
    # Calculate the times of appearance for each loyalty_number
    loyalty_number_value_counts = df_customer_flights['loyalty_number'].value_counts()
    print (f"\n>>> VALUES COUNT:")
    print(loyalty_number_value_counts)
    # filter the Series to keep only the values that appear more than 24 times
    values_to_keep = loyalty_number_value_counts[loyalty_number_value_counts > 24].index
    print (f"\n>>> VALUES WITH MORE THAN 24 ROWS:")
    print(values_to_keep)

In [ ]:
filtered_df = df_customer_flights[df_customer_flights['loyalty_number'].isin(values_to_keep)].sort_values(by=['loyalty_number','year', 'month','total_flights'])
filtered_df.head(20)

In [ ]:
df_customer_loyalty[df_customer_loyalty['loyalty_number'].isin(values_to_keep)]